In [27]:
import pandas as pd, numpy as np
from sklearn.ensemble import IsolationForest
from scipy.stats import skew, kurtosis
import numpy as np
import plotly.express as px


In [28]:
def iqr_approach(series, feature):
    summary = series.describe()
    return pd.DataFrame({
        'feature': [feature],
        'lower_bound_iqr': round(summary['25%'] - 1.5 * (summary['75%'] - summary['25%']), 2),
        'upper_bound_iqr': round(summary['75%'] + 1.5 * (summary['75%'] - summary['25%']), 2)
    })

def std_approach(series, feature, alpha: int = 3):
    summary = series.describe()
    return pd.DataFrame({
        'feature': [feature],
        'lower_bound_std': round(summary['mean'] - alpha * summary['std'], 2),
        'upper_bound_std': round(summary['mean'] + alpha * summary['std'], 2)
    })

def modified_z_score_approach(series, feature, alpha: int = 3.5):
    median = series.median()
    absolute_deviations = np.abs(series - median)
    mad = absolute_deviations.median()

    if mad == 0:
        lower_bound = upper_bound = median
    else:
        lower_bound = round(median - alpha * mad, 2)
        upper_bound = round(median + alpha * mad, 2)

    return pd.DataFrame({
        'feature': [feature],
        'lower_bound_mod_z_score': lower_bound,
        'upper_bound_mod_z_score': upper_bound
    })

def f1(z: pd.Series, beta_1: int, beta_2: int) -> pd.Series:
    anomaly_mask = (z <= (-beta_1)) | (z >= beta_2)
    return anomaly_mask

def f2(z: pd.Series, beta_1: int, beta_2: int, gamma: int = 3) -> pd.Series:
    isolation_forest = IsolationForest()
    mask_f1 = f1(z, gamma * beta_1, gamma * beta_2)
    isolation_forest.fit(z.values.reshape(-1, 1))  
    mask_iforest = isolation_forest.predict(z.values.reshape(-1, 1)) == -1 
    return mask_f1 & mask_iforest

def gamma_outlier(series, feature, alphas: int = 6, alphak: int = 30):
    
    mean = series.mean()
    std_dev = series.std()

    z_scores = (series - mean) / std_dev

    skew_result, kurt_result = abs(skew(z_scores)), abs(kurtosis(z_scores))
    
    if skew_result < alphas and kurt_result < alphak:
        mask = f1(z_scores, beta_1=3, beta_2=3)
    else:
        mask = f2(z_scores, beta_1=3, beta_2=3, gamma=2)

    filtered_data = series[~mask]

    if len(filtered_data) > 0:
        return pd.DataFrame({
            'feature': [feature],
            'lower_bound_gamma': round(filtered_data.min(), 2),
            'upper_bound_gamma': round(filtered_data.max(), 2)
        })

def jaccard_index(lower_a, upper_a, lower_b, upper_b):
    overlap = max(0, min(upper_b, upper_a) - max(lower_b, lower_a))
    distance = max(0, max(upper_b, upper_a) - min(lower_b, lower_a))
    
    return overlap / distance if distance != 0 else 0



In [29]:
np.random.seed(42)

num_features = 10  
num_data_points = 100  
num_anomalies = 5  


data = {}
min_values = []
max_values = []

for i in range(1, num_features + 1):
    feature_name = f'Feature_{i}'
    normal_data = np.random.normal(loc=10, scale=5, size=num_data_points)
    min_values.append(np.min(normal_data))  
    max_values.append(np.max(normal_data))  
    anomalies = np.random.uniform(low=50, high=100, size=num_anomalies)
    combined_data = np.concatenate([normal_data, anomalies]) 
    data[feature_name] = combined_data
features_list = []
values_list = []

for feature in data:
    features_list.append(feature)
    values_list.append(data[feature].tolist()) 

final_df = pd.DataFrame({
    'feature': features_list,
    'values': values_list,
    'lower': min_values,
    'upper': max_values
})

final_df

,feature,values,lower,upper
0,Feature_1,"[12.483570765056164, 9.308678494144077, 13.238...",-3.098726,19.261391
1,Feature_2,"[11.074694150473775, -0.11157466597867582, 5.2...",-0.111575,23.672111
2,Feature_3,"[15.415256215876385, 15.269010260174515, 3.111...",-6.206337,29.263657
3,Feature_4,"[6.653986359093935, 15.809686588733873, 17.764...",-2.261323,21.378806
4,Feature_5,"[15.896485920319133, 10.337592407050545, 20.30...",-1.509606,25.394404
5,Feature_6,"[7.177214983652124, 4.989766381856329, 11.3987...",-3.704774,23.918887
6,Feature_7,"[10.278624561443474, 15.470957592354743, 1.537...",-3.484433,22.866799
7,Feature_8,"[1.8520350701183634, 12.759318276547504, 20.02...",-2.423058,20.406573
8,Feature_9,"[14.921611992382918, 8.930055778872456, 9.7526...",-0.994030,22.634662
9,Feature_10,"[8.887936107722172, 4.265206856332804, 9.42704...",2.042977,24.825052


In [30]:
# Initialize empty DataFrames for filter ranges
filter_ranges_iqr = []
filter_ranges_std = []
filter_ranges_modified_z = []
filter_ranges_gamma = []

# Iterate over each row in the final DataFrame
for index, row in final_df.iterrows():
    # Extract the 'Values' column as a Series
    values_series = row['values']
    feature = row['feature']
    # Flatten the lists into a single Series
    #values_series = values_series.explode()

    # Convert to float if necessary
    values_series = pd.Series(values_series)

    # Apply each outlier detection method on the Series
    filter_row_iqr = iqr_approach(values_series, feature)
    filter_row_std = std_approach(values_series, feature)
    filter_row_modified_z = modified_z_score_approach(values_series, feature)
    filter_row_gamma = gamma_outlier(values_series, feature)

    filter_ranges_iqr.append(filter_row_iqr)
    filter_ranges_std.append(filter_row_std)
    filter_ranges_modified_z.append(filter_row_modified_z)
    filter_ranges_gamma.append(filter_row_gamma)
    

In [31]:
filter_ranges_iqr = pd.concat(filter_ranges_iqr, ignore_index=True)

In [32]:
filter_ranges_std = pd.concat(filter_ranges_std, ignore_index=True)

In [33]:
filter_ranges_modified_z = pd.concat(filter_ranges_modified_z, ignore_index=True)

In [34]:
filter_ranges_gamma = pd.concat(filter_ranges_gamma, ignore_index=True)

In [35]:
filter_ranges_iqr

,feature,lower_bound_iqr,upper_bound_iqr
0,Feature_1,-1.62,21.86
1,Feature_2,-3.77,25.29
2,Feature_3,-4.54,25.52
3,Feature_4,-2.42,25.26
4,Feature_5,-6.43,25.90
5,Feature_6,-3.13,24.51
6,Feature_7,-2.18,23.01
7,Feature_8,-6.11,26.12
8,Feature_9,-4.28,27.10
9,Feature_10,-0.98,22.37


In [36]:
dfs = [filter_ranges_iqr, filter_ranges_std, filter_ranges_modified_z, filter_ranges_gamma]

for df in dfs:
    final_df = pd.merge(final_df, df, on=['feature'], how='left')

In [37]:
final_df

,feature,values,lower,upper,lower_bound_iqr,upper_bound_iqr,lower_bound_std,upper_bound_std,lower_bound_mod_z_score,upper_bound_mod_z_score,lower_bound_gamma,upper_bound_gamma
0,Feature_1,"[12.483570765056164, 9.308678494144077, 13.238...",-3.098726,19.261391,-1.62,21.86,-29.98,54.74,-0.08,19.72,-3.10,19.26
1,Feature_2,"[11.074694150473775, -0.11157466597867582, 5.2...",-0.111575,23.672111,-3.77,25.29,-35.31,63.28,-2.98,24.01,-0.11,57.00
2,Feature_3,"[15.415256215876385, 15.269010260174515, 3.111...",-6.206337,29.263657,-4.54,25.52,-36.49,63.87,-2.60,23.73,-6.21,29.26
3,Feature_4,"[6.653986359093935, 15.809686588733873, 17.764...",-2.261323,21.378806,-2.42,25.26,-30.24,57.84,-0.81,23.60,-2.26,55.68
4,Feature_5,"[15.896485920319133, 10.337592407050545, 20.30...",-1.509606,25.394404,-6.43,25.90,-35.81,61.98,-4.34,23.71,-1.51,25.39
5,Feature_6,"[7.177214983652124, 4.989766381856329, 11.3987...",-3.704774,23.918887,-3.13,24.51,-35.53,62.48,-1.04,22.69,-3.70,56.23
6,Feature_7,"[10.278624561443474, 15.470957592354743, 1.537...",-3.484433,22.866799,-2.18,23.01,-27.85,52.17,-2.00,21.46,-3.48,50.92
7,Feature_8,"[1.8520350701183634, 12.759318276547504, 20.02...",-2.423058,20.406573,-6.11,26.12,-34.53,60.14,-3.06,24.40,-2.42,54.47
8,Feature_9,"[14.921611992382918, 8.930055778872456, 9.7526...",-0.994030,22.634662,-4.28,27.10,-30.99,59.57,-2.41,24.90,-0.99,54.12
9,Feature_10,"[8.887936107722172, 4.265206856332804, 9.42704...",2.042977,24.825052,-0.98,22.37,-28.80,56.04,0.28,20.46,2.04,52.01


In [38]:
# Calculate Jaccard index for each feature
jaccard_results = {
                    'feature':[],
                    'IQR':[],
                   'STD': [],
                   'MOD_Z': [],
                   'GAMMA':[]
                   }

for method_iters in range(5):
    for index, row in final_df.iterrows():
        match method_iters:
            case 0:
                jaccard_results['feature'].append(row['feature'])
            case 1:
                lower= row['lower']
                upper = row['upper']
                jaccard_value = jaccard_index(lower_a=row['lower_bound_iqr'],upper_a=row['upper_bound_iqr'], lower_b=lower, upper_b=upper)
                jaccard_results['IQR'].append(jaccard_value)
            case 2:
                lower= row['lower']
                upper = row['upper']
                jaccard_value = jaccard_index(lower_a=row['lower_bound_std'],upper_a=row['upper_bound_std'], lower_b=lower, upper_b=upper)
                jaccard_results['STD'].append(jaccard_value)
            case 3:
                lower= row['lower']
                upper = row['upper']
                jaccard_value = jaccard_index(lower_a=row['lower_bound_mod_z_score'],upper_a=row['upper_bound_mod_z_score'], lower_b=lower, upper_b=upper)
                jaccard_results['MOD_Z'].append(jaccard_value)
            case 4:
                lower= row['lower']
                upper = row['upper']
                jaccard_value = jaccard_index(lower_a=row['lower_bound_gamma'],upper_a=row['upper_bound_gamma'], lower_b=lower, upper_b=upper)
                jaccard_results['GAMMA'].append(jaccard_value)


In [39]:
jaccard_results = pd.DataFrame(jaccard_results)


In [40]:
final_df = pd.merge(final_df, jaccard_results, on=['feature'], how='left')

In [41]:
final_df

,feature,values,lower,upper,lower_bound_iqr,upper_bound_iqr,lower_bound_std,upper_bound_std,lower_bound_mod_z_score,upper_bound_mod_z_score,lower_bound_gamma,upper_bound_gamma,IQR,STD,MOD_Z,GAMMA
0,Feature_1,"[12.483570765056164, 9.308678494144077, 13.238...",-3.098726,19.261391,-1.62,21.86,-29.98,54.74,-0.08,19.72,-3.10,19.26,0.836637,0.263930,0.847610,0.999881
1,Feature_2,"[11.074694150473775, -0.11157466597867582, 5.2...",-0.111575,23.672111,-3.77,25.29,-35.31,63.28,-2.98,24.01,-0.11,57.00,0.818434,0.241238,0.881204,0.416415
2,Feature_3,"[15.415256215876385, 15.269010260174515, 3.111...",-6.206337,29.263657,-4.54,25.52,-36.49,63.87,-2.60,23.73,-6.21,29.26,0.847477,0.353428,0.742318,0.999794
3,Feature_4,"[6.653986359093935, 15.809686588733873, 17.764...",-2.261323,21.378806,-2.42,25.26,-30.24,57.84,-0.81,23.60,-2.26,55.68,0.854051,0.268394,0.857992,0.407978
4,Feature_5,"[15.896485920319133, 10.337592407050545, 20.30...",-1.509606,25.394404,-6.43,25.90,-35.81,61.98,-4.34,23.71,-1.51,25.39,0.832169,0.275120,0.848162,0.999822
5,Feature_6,"[7.177214983652124, 4.989766381856329, 11.3987...",-3.704774,23.918887,-3.13,24.51,-35.53,62.48,-1.04,22.69,-3.70,56.23,0.958678,0.281845,0.859046,0.460816
6,Feature_7,"[10.278624561443474, 15.470957592354743, 1.537...",-3.484433,22.866799,-2.18,23.01,-27.85,52.17,-2.00,21.46,-3.48,50.92,0.945361,0.329308,0.890281,0.484277
7,Feature_8,"[1.8520350701183634, 12.759318276547504, 20.02...",-2.423058,20.406573,-6.11,26.12,-34.53,60.14,-3.06,24.40,-2.42,54.47,0.708335,0.241150,0.831378,0.401219
8,Feature_9,"[14.921611992382918, 8.930055778872456, 9.7526...",-0.994030,22.634662,-4.28,27.10,-30.99,59.57,-2.41,24.90,-0.99,54.12,0.752986,0.260918,0.865203,0.428651
9,Feature_10,"[8.887936107722172, 4.265206856332804, 9.42704...",2.042977,24.825052,-0.98,22.37,-28.80,56.04,0.28,20.46,2.04,52.01,0.787715,0.268530,0.750335,0.455915


In [42]:
length_df = len(final_df)

iqr_mean = final_df['IQR'].sum() / length_df
z_score_mean = final_df['STD'].sum() / length_df
mod_z_mean = final_df['MOD_Z'].sum() / length_df
gamma_mean = final_df['GAMMA'].sum() / length_df

df_eval = pd.DataFrame({
    'IQR': [iqr_mean],
    'STD': [z_score_mean],
    'MOD_Z': [mod_z_mean],
    'GAMMA': [gamma_mean]
}, index=['mean jaccard coeff'])

In [43]:
df_jaccard = final_df.iloc[:, -4:]

fig = px.imshow(df_jaccard, 
                text_auto=".2f",  
                color_continuous_scale="YlGnBu", 
                labels=dict(color="Jaccard Index")) 

fig.update_layout(
    title="Jaccard Index Heatmap",
    xaxis_title="Methods",
    yaxis_title="Features"
)


fig.show()


In [44]:
df_eval

,IQR,STD,MOD_Z,GAMMA
mean jaccard coeff,0.834184,0.278386,0.837353,0.605477


In [45]:
best_method = df_eval.idxmax(axis=1)

In [46]:
print(f'Best Method based on labeled data: {best_method.values[0]}')

Best Method based on labeled data: MOD_Z
